# 📘 ETo CIMIS Calculation – README

This notebook demonstrates how to calculate **hourly reference evapotranspiration (RET)** and **daily ETo** using the `eto_hourly_cimis` function, following the **CIMIS method** and equations from the provided flowchart.

---

## 🔧 Required Inputs

The function `eto_hourly_cimis()` requires the following inputs for each **hour**:

| Variable      | Description                               | Units     | Example Range      |
|---------------|-------------------------------------------|-----------|---------------------|
| `T_C`         | Mean hourly air temperature               | °C        | 10 to 45            |
| `ea_kPa`      | Mean hourly actual vapor pressure          | kPa       | 0.5 to 3.5          |
| `Rn_Wm2`      | Mean hourly net radiation                  | W/m²      | -50 (night) to 700  |
| `U2_ms`       | Mean hourly wind speed at 2 meters         | m/s       | 0.1 to 6            |
| `Z_m`         | Elevation above mean sea level             | m         | 0 to 3000           |
| `time_index`  | Hourly timestamps (optional, for plotting) | datetime  | e.g., hourly DatetimeIndex |

---

## 📤 Outputs

If `time_index` is provided, the function returns:

- `df_hourly`: a DataFrame with hourly RET and intermediate variables.
- `eto_daily`: a Series of daily ETo totals (mm/day), computed by summing 24 RET values per day.

---

## ✅ Output Units

- **Hourly RET** → mm/hour
- **Daily ETo** → mm/day

---

## 📁 Output Files (example names)

- `outputs/hourly_components_and_RET.csv`
- `outputs/daily_ETo.csv`

---

## 🧠 Notes

- The method follows CIMIS guidelines and matches the flowchart + formulas you provided.
- Day/night conditions are handled using Rn sign for the wind function FU2.
- Inputs can be NumPy arrays, Pandas Series, or scalars (broadcastable).



In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# Step 1: Load the Excel file
input_path = Path("Inputs") / "ec_part_data.xlsx"
df_raw = pd.read_excel(input_path)

# Step 2: Parse TIMESTAMP_END into datetime
# TIMESTAMP_END format: YYYYMMDDHHMM (e.g., 202508260030)
df_raw['time'] = pd.to_datetime(df_raw['TIMESTAMP_END'].astype(str), format='%Y%m%d%H%M')

# Step 3: Rename columns to shorter names
df = df_raw.rename(columns={
    'TA': 'T_C',    # Air temperature (°C)
    'RH': 'RH_pct', # Relative humidity (%)
    'Rn': 'Rn_Wm2',  # Net radiation (W/m²)
    'U2': 'U2_ms'   # Wind speed at 2m (m/s)
})

# Step 4: Compute vapor pressure (ea_kPa) from RH and T
# Formula: ea = RH (%) × es / 100
# where es = 0.6108 * exp((17.27 * T) / (237.3 + T))
T = df['T_C'].astype(float)
RH = df['RH_pct'].astype(float)
es = 0.6108 * np.exp((17.27 * T) / (237.3 + T))
ea_kPa = RH / 100.0 * es
df['ea_kPa'] = ea_kPa
U2 = df['U2'].astype(float)

# Step 5: Add elevation (Z = 63.4 m)
Z_m = 63.4
df['Z_m'] = Z_m

# Step 6: Set datetime as index (optional but useful)
df = df.set_index('time')

# Step 8: Preview the prepared DataFrame
df[['T_C', 'RH_pct', 'Rn_Wm2', 'ea_kPa', 'U2_ms']].head()

KeyError: 'U2'

In [6]:
# 1) Imports and path setup
import sys
from pathlib import Path

# Assume this notebook sits at project root and 'Functions/eto_cimis.py' lives in ./Functions
project_root = Path.cwd()
functions_dir = project_root / "Functions"
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
if str(functions_dir) not in sys.path:
    sys.path.insert(0, str(functions_dir))

from Functions.eto_cimis import eto_cimis

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# 2) Create demo hourly data (48 hours)
start = pd.Timestamp("2021-07-01 00:00:00")
ti = pd.date_range(start, periods=48, freq="H")  # hourly 2 days

rng = np.random.default_rng(42)

# Temperature (°C): diurnal cycle ~ 18–32
T_daily_amp = 7
T_base = 25
hours = np.arange(len(ti))
T_C = T_base + T_daily_amp * np.sin(2 * np.pi * (hours % 24) / 24.0) + rng.normal(0, 0.5, len(ti))

# Saturation vapor pressure (kPa) for generating a consistent ea
es = 0.6108 * np.exp((17.27 * T_C) / (237.3 + T_C))

# Relative humidity ~ 35–55% (random)
RH = rng.uniform(35, 55, len(ti)) / 100.0
ea_kPa = RH * es

# Net radiation Rn (W/m^2): positive daytime, negative small at night
day_factor = (np.sin(2 * np.pi * (hours % 24) / 24.0) > 0).astype(float)
Rn_day = 500 * np.sin(2 * np.pi * (hours % 24) / 24.0).clip(min=0)
Rn_night = -50 * (1 - day_factor)
Rn_Wm2 = Rn_day + Rn_night + rng.normal(0, 10, len(ti))

# Wind speed at 2 m (m/s)
U2_ms = np.clip(rng.normal(2.5, 0.8, len(ti)), 0.1, None)

# Elevation (m)
Z_m = 100.0

C:\Users\Raymond G\AppData\Local\Temp\ipykernel_45392\57318631.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ti = pd.date_range(start, periods=48, freq="H")  # hourly 2 days


In [8]:
# 3) Run the function
df_hourly, eto_daily = eto_hourly_cimis(
    T_C=T_C,
    ea_kPa=ea_kPa,
    Rn_Wm2=Rn_Wm2,
    U2_ms=U2_ms,
    Z_m=Z_m,
    time_index=ti,
)
df_hourly.head(), eto_daily.head()

NameError: name 'eto_hourly_cimis' is not defined

In [ ]:
# 4) Quick visualization (matplotlib, single-figure as requested)
plt.figure(figsize=(10, 4))
eto_daily.plot(marker='o')
plt.title('Daily ETo (mm/day) from hourly RET')
plt.xlabel('Date')
plt.ylabel('ETo (mm/day)')
plt.grid(True)
plt.show()

In [ ]:
# 5) Save outputs
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
df_hourly.to_csv(out_dir / "hourly_components_and_RET.csv")
eto_daily.to_csv(out_dir / "daily_ETo.csv", header=True)
print("Saved:", out_dir / "hourly_components_and_RET.csv")
print("Saved:", out_dir / "daily_ETo.csv")

### (Optional) Save to NetCDF with `xarray`
If you want to keep coordinates/dimensions and write NetCDF:

```python
import xarray as xr

ds_hourly = xr.Dataset({
    'RET_mm_hr': (['time'], df_hourly['RET_mm_hr'].values),
    'T_C': (['time'], df_hourly['T_C'].values),
    'ea_kPa': (['time'], df_hourly['ea_kPa'].values),
    'Rn_Wm2': (['time'], df_hourly['Rn_Wm2'].values),
    'U2_ms': (['time'], df_hourly['U2_ms'].values),
}, coords={'time': df_hourly.index})
ds_hourly.to_netcdf('outputs/hourly_RET.nc')

ds_daily = xr.Dataset({
    'ETo_mm_day': (['date'], eto_daily.values)
}, coords={'date': eto_daily.index})
ds_daily.to_netcdf('outputs/daily_ETo.nc')
```